# Part 17: Analysis of scRNAseq data from Honardoost et al., 2024

This is a reanalysis of data published in [Honardoost et al., 2024](https://genomemedicine.biomedcentral.com/articles/10.1186/s13073-024-01300-z). We work with the Seurat object provided by the authors of the study. 

In [ ]:
source("diabetes_analysis_v07.R")


Let's load the downloaded object. If you are trying to recapitulate the analysis, please download the object from the Synapse database: [syn53641849](https://www.synapse.org/#!Synapse:syn53641849). The downloaded object should be saved in the folder: `../data/published_data/Honardoost_2024/`. 

In [ ]:
honardoost  <- readRDS("../data/published_data/Honardoost_2024/T1D_Seurat_Object_Final.rds")

Process data using our pipeline. 

In [ ]:
honardoost <- NormalizeData(honardoost, verbose = FALSE)
honardoost <- FindVariableFeatures(honardoost, nfeatures = 1000, verbose = FALSE)
honardoost <- ScaleData(honardoost, verbose = FALSE)
honardoost <- RunPCA(honardoost, npcs = 20, verbose = FALSE)
honardoost <- RunUMAP(honardoost, reduction = "pca", dims = 1:20)

In [ ]:
honardoost <- FindNeighbors(honardoost, dims = 1:12)
honardoost <- FindClusters(honardoost, resolution = 0.3)

Keep only T-cell clusters. 

In [ ]:
honardoostT  <- subset(honardoost, seurat_clusters %in% c(0,1,3,5,6))

In [ ]:
honardoostT <- NormalizeData(honardoostT, verbose = FALSE)
honardoostT <- FindVariableFeatures(honardoostT, nfeatures = 1000, verbose = FALSE)
honardoostT <- ScaleData(honardoostT, verbose = FALSE)
honardoostT <- RunPCA(honardoostT, npcs = 20, verbose = FALSE)
honardoostT <- RunUMAP(honardoostT, reduction = "pca", dims = 1:20)
honardoostT <- FindNeighbors(honardoostT, dims = 1:12)
honardoostT <- FindClusters(honardoostT, resolution = 0.3)

In [ ]:
saveRDS(honardoostT, "../data/published_data/Honardoost_2024/honardoost_tcells.rds")

# Analysis of T cells

In [ ]:
honardoostT  <- readRDS("../data/published_data/Honardoost_2024/honardoost_tcells.rds")

Annotations provided by authors. 

In [ ]:
DimPlot(honardoostT, raster = T, label = T, group.by = "Cluster_Annotation_Merged")

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 16)

DimPlot(honardoostT, raster = T, label = T, group.by = "Sample_ID")

In [ ]:
honardoostT <- FindClusters(honardoostT, resolution = 0.23)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)
DimPlot(honardoostT, raster = T, label = T)

Clustering and cluster markers. 

In [ ]:
mrk  <- FindAllMarkers(honardoostT, logfc.threshold = log(2))

In [ ]:
mrk  %>% filter(cluster ==0 & avg_log2FC >0)

In [ ]:
options(repr.plot.width = 20)
VlnPlot(honardoostT, features = c("percent.mt", "percent.rp", "nCount_RNA", "nFeature_RNA"), ncol = 4, raster = F)

In [ ]:
options(repr.plot.width = 6)

FeaturePlot(honardoostT, features = "FOXP3", min.cutoff = 0, raster = T, max.cutoff = 1)


We see that many cluster markers are unwanted genes. We will proceed with filtering out unwanted genes, such as mitochondrial and ribosomal genes, unannotated genes etc. 

In [ ]:
mtx  <- honardoostT@assays$RNA@counts[which(!(grepl(rownames(honardoostT@assays$RNA@counts), pattern = "MT-")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "\\.")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "^MIR")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "^MT[RN]")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "-AS")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "LINC")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "ORF")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "^TR[AB]")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "^RP[LS]")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "\\-")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "\\.")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "mir")) ),]

Create new Seurat object. 

In [ ]:
honar2  <- CreateSeuratObject(counts = mtx, meta.data = honardoostT@meta.data, min.cells = 3, min.features = 200)

In [ ]:
honar2 <- NormalizeData(honar2, verbose = FALSE)
honar2 <- FindVariableFeatures(honar2, nfeatures = 1000, verbose = FALSE)
honar2 <- ScaleData(honar2, verbose = FALSE)
honar2 <- RunPCA(honar2, npcs = 20, verbose = FALSE)
honar2 <- RunUMAP(honar2, reduction = "pca", dims = 1:20)
honar2 <- FindNeighbors(honar2, dims = 1:12)
honar2 <- FindClusters(honar2, resolution = 0.3)

In [ ]:
DimPlot(honar2, group.by = "Cluster_Annotation_Merged")

In [ ]:
DimPlot(honar2)

New markers. 

In [ ]:
mrk  <- FindAllMarkers(honar2)

In [ ]:
options(repr.plot.height = 15, repr.plot.width = 20)
for(i in 0:6){
    print(paste("Cluster",i))
    genes  <- mrk  %>% dplyr::filter(cluster == i)  %>% slice_head(n = 12)  %>% pull(gene)
    print(FeaturePlot(honar2, features = genes, ncol = 4))
    }

# STACAS

Remove batch effect by integration using STACAS. 

In [ ]:
honar2  <- SplitObject(honar2, split.by = "LIB")

In [ ]:
# normalize and identify variable features for each dataset independently
merged.list <- lapply(X = honar2, FUN = function(x) {
    DefaultAssay(x)  <- "RNA"
    x$barcode  <- colnames(x)
    x <- NormalizeData(x)
    x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 800)
})

honar2_stacas <- Run.STACAS(merged.list, dims = 1:12)
honar2_stacas <- RunUMAP(honar2_stacas, dims = 1:12) 

In [ ]:
DimPlot(honar2_stacas)

In [ ]:
DimPlot(honar2_stacas, group.by = "Cluster_Annotation_Merged")

In [ ]:
DimPlot(honar2_stacas, group.by = "Cluster_Annotation_All")

In [ ]:
FeaturePlot(honar2_stacas, features = "FOXP3", min.cutoff = 0)
FeaturePlot(honar2_stacas, features = "CD226", min.cutoff = 0)
FeaturePlot(honar2_stacas, features = "CCR7", min.cutoff = 0)
FeaturePlot(honar2_stacas, features = "TNFRSF9", min.cutoff = 0)
FeaturePlot(honar2_stacas, features = "CCR10", min.cutoff = 0)

In [ ]:
FeaturePlot(honar2_stacas, features = "MKI67", min.cutoff = 0)

In [ ]:
saveRDS(honar2_stacas, "../data/published_data/Honardoost_2024/honar2_stacas.rds")